In [1]:
import numpy as np

In [2]:
'''-------- Data Preprocessing  --------'''

'''-------- Cleaning of Sentences --------'''

def funtuationMarksSep (Sentence):    
     
    strippedSen = Sentence.lstrip().rstrip()
    dots =[i for i, letter in enumerate(strippedSen) if letter == '.']
    
    if (len(dots)) > 0:
        strippedSen = strippedSen.replace('.', '')
        strippedSen = strippedSen + ' .'
    else:
        strippedSen = strippedSen + ' .'    
    
    commas = [i for i, letter in enumerate(strippedSen) if letter == ',']
    if (len(commas)) > 0:
        sen = []

        for i in range (len(commas)):
            if ( i == 0):
                sen.append(strippedSen[:(commas[i])])
            else:
                sen.append(strippedSen[(commas[i-1]):(commas[i])])
        
            if (i == (len(commas)) -1):
                sen.append(strippedSen[(commas[i]):])    

        sentence = ''
        
        for i in range (len(sen)):
            pos = sen[i].find (',')    
    
            if (pos > 0):
                sen[i] = sen[i][:pos]       
                sentence = sentence.lstrip().rstrip() + ' '+ sen[i].lstrip().rstrip()   
                
            elif (pos == 0):
                sen[i] = sen[i][pos+1:]         
                sentence =  sentence.lstrip().rstrip() + ' , '+ sen[i].lstrip().rstrip() 
                
            else:
                if not (sen[i] == ' '):
                    sentence = sentence.lstrip().rstrip()+ ' ' + sen[i].lstrip().rstrip()
                    
    else:
        sentence = strippedSen
        
    return sentence

In [3]:
'''-------- Data Preprocessing  --------'''

'''-------- Reading & Loading the Synonyms --------'''

def synonymWordLoading ():
    
    synonymWords = []
    synonymWordFile = "similarWords.txt"

    with open(synonymWordFile,"r") as data:
    
        for line in data:
            words = line.split('\t') 
            wordList = []    
            lngth = len(words)
                
            if (words[lngth - 1] == '\n') :        
            
                words.pop(0)            
                length = len(words)
                words.pop(length-1)
                words.pop(length-2)
                words.remove('-')
                wordList.append(words[0].lstrip().rstrip())
            
                if (len(words) == 1):
                    wordList.append(words[0])
                else:
                    newWord = words[1].split(',')           
            
                    for i in range (len(newWord)):
                        wordList.append(newWord[i].lstrip().rstrip()) 
            else :
                words.pop(0)
                words.remove('-') 
                wordList.append(words[0].lstrip().rstrip())
                words = words[1].replace('\n', '')   
            
                if (len(words) == 1):                
                    wordList.append(words[0])
                else:
                    newWord = words.split(',')           
            
                    for i in range (len(newWord)):                    
                        wordList.append(newWord[i].lstrip().rstrip())            
            
            synonymWords.append(wordList)
            
    return synonymWords

In [4]:
'''-------- Data Preprocessing  --------'''

'''-------- Replacing Synonyms --------'''

def synonymWordReplacement(Sentence,synonymWords):
    
    newWords = []
    
    words = Sentence.split(" ") 
    sen = ''
    
    for i in range (len(words)):
        wordExsitence = True         
                
        for j in range (len(synonymWords)):
            synonymList = synonymWords[j]  
                                               
            if ((words[i].lstrip().rstrip()) == synonymList[0] ):                 
                newWords.append(synonymList[0])                               
                wordExsitence = False
                break
                
            elif (words[i].lstrip().rstrip()) in synonymList[1:(len(synonymList))]:                
                newWords.append(synonymList[0])
                wordExsitence = False
                break
                        
        if (wordExsitence):
            newWords.append(words[i].lstrip().rstrip()) 
    
    newSen = ''
    
    for i in range (len (newWords)) :
        if (i == 0):
            newSen = newWords[i] + " "
        else :
            newSen = newSen + newWords[i] + " " 
    
                
    return newSen, newWords  

In [5]:
'''-------- Data Preprocessing  --------'''

'''-------- Stemming Of Words --------'''

def wordStemming(sentence, wordVector):
            
    rows = []
    words = []    
    
    for token in sentence:
        i = len(token)
        
        while len(token) > 0 and i > 0:
            word = token[:i]
            
            if word in wordVector:  
                
                rows.append(wordVector[word])
                words.append(word)
                token = token[i:]
                i = len(token)
            else:
                i = i-1
                
    return rows, words


In [6]:
'''-------- Data Preprocessing --------'''

'''-------- Data Preprocessing Function--------'''

def TrainDataPreProcessing (synonymWords, fileName):
    
    eviSentences = []
    hypSentences = []
    labels = []
    #scores = [] 
    DataSet = []
    cnt = 0
        
    with open(fileName,"r") as data:
        for line in data:  
            cnt =  cnt + 1    
            records = line.split('|') 
            
            if (len(records) > 1):                
                Sentence = []
                cleanedEviSentence = funtuationMarksSep(records[1].lstrip().rstrip())
                replacedEviSentence, eviSen = synonymWordReplacement ((cleanedEviSentence.lstrip().rstrip()), synonymWords)               
                eviSentences.append(eviSen )               
                
                cleanedHypSentence = funtuationMarksSep(records[2].lstrip().rstrip())                
                replacedHypSentence, hypSen = synonymWordReplacement ((cleanedHypSentence.lstrip().rstrip()), synonymWords) 
                hypSentences.append(hypSen)                
                
                labels.append((records[0].lstrip()).rstrip())                
                #scores.append(scoreSetup(records[3].lstrip().rstrip())) 
                Sentence.append(replacedEviSentence)
                Sentence.append(replacedHypSentence)
                Sentence.append((records[0].lstrip()).rstrip())
                                
                DataSet.append (Sentence)   
              
        return DataSet, eviSentences, hypSentences, labels
    

In [8]:
'''-------- Data Preprocessing --------'''

'''-------- Data Loading & Preprocessing --------'''

trainDataFileName = "Data/trainData.txt"
valDataFileName = "Data/valData.txt"
testDataFileName = "Data/testData.txt"

synonymWords = synonymWordLoading ()

processedTrainDataSet, trainEviSen, trainHypSen, trainLabels = TrainDataPreProcessing (synonymWords, trainDataFileName)
processedValidationDataSet, valEviSen, valHypSen, valLabels = TrainDataPreProcessing (synonymWords, valDataFileName)
processedTestDataSet, testEviSen, testHypSen, testLabels = TrainDataPreProcessing (synonymWords, testDataFileName)

print (len(processedTrainDataSet))
print (len(processedValidationDataSet))
print (len(processedTestDataSet))

11360
1245
469


In [10]:
'''-------- Neural Network --------'''

'''-------- Imports--------'''

import torch
import torchtext
from torchtext import data
from torchtext import datasets
import torch.optim as optim
import spacy

In [11]:
'''-------- Neural Network --------'''

'''-------- Toeknizer --------'''


Toeknizer = spacy.load('en')

def tokenizer(text):
    return [tok.text for tok in Toeknizer.tokenizer(text)]

TEXT = data.Field(sequential=True, tokenize=tokenizer, lower=True, batch_first= True)
print (type(TEXT))

LABEL = data.LabelField(dtype=torch.long, batch_first= True)
print (type(LABEL))

Splitter = [('Evi',TEXT), ('Hyp',TEXT),('Label',LABEL)]

<class 'torchtext.data.field.Field'>
<class 'torchtext.data.field.LabelField'>


In [12]:
'''-------- Neural Network --------'''

'''-------- Dataset Preparation --------'''

Example = data.Example()
trainSentences=[]
validationSentences = []
testSentences=[]

for i in range (len(processedTrainDataSet)):
    trainSentences.append(Example.fromlist(processedTrainDataSet[i], Splitter))
    
trainDS = data.Dataset(trainSentences, Splitter)

for i in range (len(processedValidationDataSet)):
    validationSentences.append(Example.fromlist(processedValidationDataSet[i], Splitter))
    
ValidationDS = data.Dataset(validationSentences, Splitter)

for i in range (len(processedTestDataSet)):
    testSentences.append(Example.fromlist(processedTestDataSet[i], Splitter))
    
testDS = data.Dataset(testSentences, Splitter)

print (type(trainDS))
print (vars(trainDS.examples[0]))
print (len(trainDS))
print ("\n")

print (type(ValidationDS))
print (vars(ValidationDS.examples[0]))
print (len(ValidationDS))
print ("\n")

print (type(testDS))
print (vars(testDS.examples[0]))
print (len(testDS))


<class 'torchtext.data.dataset.Dataset'>
{'Evi': ['ප්\u200dරෝටීන', 'යනු', 'ඇමයිනෝ', 'ඇඹුල්', 'දාම', 'එකකින්', 'හෝ', 'වැඩි', 'ගණනකින්', 'සැදුම්ලත්', 'විශාල', 'කාබනික', 'සංයෝගයන්ය', '.'], 'Hyp': ['යනු', 'හෝ', 'සංයෝගයන්ය', 'සැදුම්ලත්', 'වැඩි', 'ඇමයිනෝ', 'දාම', 'ප්\u200dරෝටීන', 'කාබනික', 'එකකින්', 'එකම', '.'], 'Label': 'P'}
11360


<class 'torchtext.data.dataset.Dataset'>
{'Evi': ['එමගින්', 'කලාපයේ', 'ජනයා', 'ගැටුම්', 'වලින්', 'තොර', 'මනා', 'කලා', 'රසඥතාවයකින්', 'හෙබි', 'සාමකාමී', 'අය', 'වුහ', '.'], 'Hyp': ['වලින්', 'මනා', 'වුහ', 'හෙබි', 'කලා', 'අය', 'කලාපයේ', 'තොර', 'සාමකාමී', 'රසඥතාවයකින්', 'ගැටුම්', 'ජනයා', 'එමගින්', 'එකඟ', '.'], 'Label': 'P'}
1245


<class 'torchtext.data.dataset.Dataset'>
{'Evi': ['ශ්\u200dරී', 'ලංකාව', 'දියුණු', 'වෙමින්', 'පවතින', 'තුන්වෙනි', 'ලෝකයේ', 'රටකි', '.'], 'Hyp': ['ශ්\u200dරී', 'ලංකාව', 'නොදියුණු', 'තුන්වෙනි', 'ලෝකයේ', 'රටකි', '.'], 'Label': 'P'}
469


In [13]:
'''-------- Neural Network --------'''

'''-------- Fast Text Embedding Loading --------'''

#file needs to downloaded from https://fasttext.cc/docs/en/crawl-vectors.html

wordVectors = torchtext.vocab.Vectors('cc.si.300.vec', cache='Downloads/')

In [14]:
'''-------- Neural Network --------'''

'''-------- Embedding Vocabulary --------'''


TEXT.build_vocab(trainDS, vectors=wordVectors)
LABEL.build_vocab(trainDS)
pretrained_embeddings = TEXT.vocab.vectors

print (len(TEXT.vocab))
print(TEXT.vocab.freqs.most_common(20))

print (len(LABEL.vocab))
print(LABEL.vocab.freqs.most_common(3))

5613
[('.', 22720), ('සහ', 5422), (',', 5170), ('අතර', 2638), ('මෙම', 2444), ('ඇති', 2302), ('වන', 2198), ('වෙනස්', 2162), ('වගේ', 2154), ('ඇත', 1984), ('වේ', 1236), ('කරන', 1234), ('විශාල', 1184), ('එය', 1184), ('මද්\u200dයස්ථ', 1180), ('මැද', 1172), ('පළමු', 1158), ('වැඩි', 1118), ('මැදහත්', 1112), ('වූ', 1052)]
3
[('C', 3796), ('P', 3784), ('N', 3780)]


In [16]:
'''-------- Neural Network --------'''

'''-------- Iterators --------'''


BATCH_SIZE = 64

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

train_iterator, validation_iterator, test_iterator = data.BucketIterator.splits(
    (trainDS,ValidationDS,testDS),sort_key=lambda x: len(x.Evi), batch_size=BATCH_SIZE, device=device)

In [17]:
'''-------- Neural Network --------'''

'''-------- Neural Network Declarion --------'''


import torch.nn as nn

class RNN(nn.Module):
    def __init__(self, vocab_size, embedding_dim, hidden_dim, output_dim, n_layers, bidirectional, dropout):
        super().__init__()
        
        self.embedding = nn.Embedding(vocab_size, embedding_dim)
        
        self.Rnn = nn.LSTM(embedding_dim, hidden_dim, num_layers=n_layers, bidirectional=bidirectional, dropout=dropout, batch_first= True)
        self.RNNFc = nn.Linear(hidden_dim*2, output_dim)        
        self.finalFc = nn.Linear(hidden_dim*4, output_dim) 
        self.RELULayer = nn.ReLU()
        
        self.dropout = nn.Dropout(dropout)
        self.cnt = 0
        
    def forward(self, EviSen, HypSen):
        
        eviEmbed = self.embedding(EviSen)        
        hypEmbed = self.embedding(HypSen)    
        
        eviOutput, (eviHidden, eviCell) = self.Rnn(eviEmbed)
        hypOutput, (hypHidden, hypCell) = self.Rnn(hypEmbed)
        
        hiddenEvi = torch.cat((eviHidden[-2,:,:], eviHidden[-1,:,:]), dim=1)
        hiddenHyp = torch.cat((hypHidden[-2,:,:], hypHidden[-1,:,:]), dim=1)
        
                
        combinedOutput = torch.cat((hiddenEvi,hiddenHyp),dim=1)        
        finalOutPut = self.finalFc(combinedOutput)  
        
        return finalOutPut


In [20]:
INPUT_DIM = len(TEXT.vocab)
EMBEDDING_DIM = 300
HIDDEN_DIM = 128
OUTPUT_DIM = 3
N_LAYERS = 2
BIDIRECTIONAL = True
DROPOUT = 0.5

model = RNN(INPUT_DIM, EMBEDDING_DIM, HIDDEN_DIM, OUTPUT_DIM, N_LAYERS, BIDIRECTIONAL, DROPOUT)
model.embedding.weight.data.copy_(pretrained_embeddings)
print(pretrained_embeddings.shape)

optimizer = optim.Adam(model.parameters()) #Optimizer
lossFun = nn.CrossEntropyLoss() #Loss Function

model = model.to(device)
criterion = lossFun.to(device)

print (model)

torch.Size([5613, 300])
RNN(
  (embedding): Embedding(5613, 300)
  (Rnn): LSTM(300, 128, num_layers=2, batch_first=True, dropout=0.5, bidirectional=True)
  (RNNFc): Linear(in_features=256, out_features=3, bias=True)
  (finalFc): Linear(in_features=512, out_features=3, bias=True)
  (RELULayer): ReLU()
  (dropout): Dropout(p=0.5)
)


In [21]:
def Accuracy(preds, correctLabels):    
    _, predicted = torch.max(preds.data, 1)
    correctPreds = 0
    CPreds = 0
    
    
    for i in range (len(predicted)):
        if predicted[i] == correctLabels[i]:
            CPreds  = CPreds  + 1
           
    #correctPreds += (predicted == correctLabels).sum().item()
    
    correctPreds = (predicted  == correctLabels).float() #convert into float for division 
    acc = correctPreds.sum()/len(correctPreds)
    
    #print ('CPreds is ' + str (CPreds))
    #print ('Percentage :' + str (CPreds/ len(correctLabels)))
    
    return acc

In [22]:
def train(model, iterator, optimizer, criterion):
    
    epoch_loss = 0
    epoch_acc = 0
    
    model.train()
    
    cnt = 0
    
    for batch in iterator:
        
        optimizer.zero_grad()
        
        predictions = model(batch.Evi, batch.Hyp) 
        acc = Accuracy(predictions, batch.Label)
        
        loss = criterion(predictions, batch.Label)               
        loss.backward()   
        
        optimizer.step()
        
        epoch_loss += loss.item()
        epoch_acc += acc.item()
        
        cnt = cnt + 1
        
    return epoch_loss / len(iterator), epoch_acc / len(iterator)
        
        

In [23]:
def validate (model, iterator, criterion):
    
    epoch_loss = 0
    epoch_acc = 0
    
    model.eval()
    
    with torch.no_grad():
    
        for batch in iterator:

            predictions = model(batch.Evi, batch.Hyp)
            
            loss = criterion(predictions, batch.Label)
            
            acc = Accuracy(predictions, batch.Label)

            epoch_loss += loss.item()
            epoch_acc += acc.item()
        
    return epoch_loss / len(iterator), epoch_acc / len(iterator)

In [24]:
N_EPOCHS = 3

for epoch in range(N_EPOCHS):

    train_loss, train_acc = train(model, validation_iterator, optimizer, criterion)
    valid_loss, valid_acc = validate(model, validation_iterator, criterion)
    #train(model, train_iterator, optimizer, criterion)
    
    print(f'| Epoch: {epoch+1:02} | Train Loss: {train_loss:.3f} | Train Acc: {train_acc*100:.2f}% | Val. Loss: {valid_loss:.3f} | Val. Acc: {valid_acc*100:.2f}% |')

| Epoch: 01 | Train Loss: 1.100 | Train Acc: 31.60% | Val. Loss: 1.095 | Val. Acc: 33.62% |
| Epoch: 02 | Train Loss: 1.082 | Train Acc: 45.49% | Val. Loss: 0.831 | Val. Acc: 62.48% |
| Epoch: 03 | Train Loss: 0.688 | Train Acc: 67.53% | Val. Loss: 0.593 | Val. Acc: 68.36% |


In [25]:
test_loss, test_acc = validate(model, test_iterator, criterion)

print(f'| Test Loss: {test_loss:.3f} | Test Acc: {test_acc*100:.2f}% |')

| Test Loss: 0.870 | Test Acc: 56.18% |
